In [1]:
import xarray as xr
import matplotlib as plt
from Chemometrics.import_common import *
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'

In [3]:
xls = pd.ExcelFile("cc11-20180428.xlsx")
df = xls.parse(0)
scan_idx = df.index[df.iloc[:,0] == 'Scan']

In [4]:
def parse_scan(scan):
    """
    解析scan
    """
    _scan = pd.Series(scan.iloc[1].values,index=scan.iloc[0]).to_dict()
    formulas = []
    for f in scan.index[scan.iloc[:,2]=='CAS #:']: # CAS #:
        _formula = {'Formula:': scan.loc[f-1 ,'Unnamed: 1'],
                    'NET FIT=': scan.loc[f, 'Unnamed: 1'],
                    'CAS #:': scan.loc[f, 'Unnamed: 3'],
                    'id': scan.loc[f-2, 'Unknown Identification Report'],
                    'Name': scan.loc[f-2, 'Unnamed: 1']}
        _formula.update(_scan)
        formulas.append(_formula.copy())
    return formulas

In [5]:
a = df.set_index('Unknown Identification Report').dropna(axis=0, how='all').reset_index()
scan_idx = (a.iloc[:,0] == 'Scan').cumsum()
scans = [a.loc[scan_idx == _sidx] for _sidx in scan_idx.unique() if _sidx > 0]
pscans = [parse_scan(_ss) for _ss in scans]
_a = []
for _ in pscans:
    _a += _
res_ = pd.DataFrame(_a)
res_.head()

,%Area,Area,CAS #:,Formula:,Hits,NET FIT=,Name,RT,Scan,id
0,0.018,2506729,75-21-8,C2H4O,1,89,Ethylene oxide,1m20s,63,1
1,0.004,1984432,107-02-8,C3H4O,7,81,2-Propenal,1m56s,97,1
2,0.004,1984432,71-36-3,C4H10O,7,75,1-Butanol,1m56s,97,2
3,0.004,1984432,106-98-9,C4H8,7,71,1-Butene,1m56s,97,3
4,0.004,1984432,590-18-1,C4H8,7,71,"2-Butene, (Z)-",1m56s,97,4


In [6]:
def get_factor_(xls_):
    """
    获取校正因子
    """
    Toluene_c = xls_.loc[(xls_ == 'Toluene').any(axis=1),xls_.isin(['ppb']).any(axis=0)]
    Toluene_area = xls_.loc[(xls_ == 'Toluene').any(axis=1),xls_.isin(['area']).any(axis=0)]
    factor_ = (Toluene_area.values/Toluene_c.values)[0]
    return factor_

In [7]:
# 计算校正因子
xls_ = pd.ExcelFile("cc11-20180428-定量.xlsx").parse(0)
factor_ = get_factor_(xls_)
res_.loc[:,'ppb'] = (res_.Area/factor_).astype(np.float)

In [9]:
grp_ = res_.groupby('CAS #:')
_out = pd.concat([grp_.first().loc[:, ['Formula:', 'Name']], grp_.sum().loc[:,['%Area', 'Area','ppb']]], axis=1).reset_index()
_out.sort_values(by = 'Area', ascending=False, inplace=True)
_out.head()

,CAS #:,Formula:,Name,%Area,Area,ppb
23,110-80-5,C4H10O2,"Ethanol, 2-ethoxy-",3.550,2846883584,10652.911533
55,6975-98-0,C11H24,"Decane, 2-methyl-",3.407,402182842,1504.950276
44,611-14-3,C9H12,"Benzene, 1-ethyl-2-methyl-",0.186,295793138,1106.844744
46,620-14-4,C9H12,"Benzene, 1-ethyl-3-methyl-",0.186,295793138,1106.844744
47,622-96-8,C9H12,"Benzene, 1-ethyl-4-methyl-",0.186,295793138,1106.844744


In [10]:
_out.to_excel('数据导入-浓度.xlsx')